In [96]:
import face_recognition
import os
from pathlib import Path
from shutil import copyfile
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import zipfile  

In [97]:
# SMTP email configuration
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
SMTP_USERNAME = 'rashisolanki2002@gmail.com'
SMTP_PASSWORD = 'ttes rjqa dlei phsh'

In [98]:
cwd = os.getcwd()
p = os.path.join(cwd, 'dataset')

In [99]:
results_path = os.path.join(cwd, 'results')
encodings = {}

In [100]:
def send_email(subject, body, to_email, attachment_path=None):
    
    msg = MIMEMultipart()
    msg['From'] = SMTP_USERNAME
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    if attachment_path:
        if os.path.isfile(attachment_path):
            attachment = open(attachment_path, 'rb')
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(attachment_path)}')
            msg.attach(part)
        else:
            print("Attachment file not found:", attachment_path)

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SMTP_USERNAME, SMTP_PASSWORD)
        text = msg.as_string()
        server.sendmail(SMTP_USERNAME, to_email, text)
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print("Error sending email:", str(e))

In [101]:
def process_user_input_image(input_image_path, email_id):
    if not os.path.isfile(input_image_path):
        print("The provided input image file does not exist.")
        return

    img = face_recognition.load_image_file(input_image_path)
    fe = face_recognition.face_encodings(img)
    if not fe:
        print("No face detected in the provided input image.")
        return

    fe = fe[0]  # Assuming only one face is detected

    cluster_found = False
    for cluster_id, cluster_encodings in encodings.items():
        results = face_recognition.compare_faces(cluster_encodings, fe)
        if all(results):
            print("The detected face belongs to cluster: %s" % cluster_id)
            cluster_found = True
            
            # Create a zip archive of the cluster directory
            email_subject = "Pictures from the event"
            email_body = "Thank you for attending the event. Here is a folder of your pictures."
            cluster_directory = os.path.join(results_path, cluster_id)
            zip_filename = f"{cluster_id}.zip"
            with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, _, files in os.walk(cluster_directory):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, cluster_directory)
                        zipf.write(file_path, arcname=arcname)
            send_email(email_subject, email_body, email_id, zip_filename)
            break

    if not cluster_found:
        print("No matching cluster found for the detected face.")

In [102]:
def process_file(filepath):
    img = face_recognition.load_image_file(filepath)
    fe = face_recognition.face_encodings(img)
    if fe:
        fe = fe[0]
    else: return
    action_taken = False
    curr_image_cluster_id = None
    for cluster_id, cluster_encodings in encodings.items():
        results = face_recognition.compare_faces(cluster_encodings, fe)
        print("results %s %s" % (results, cluster_id))
        if all(results):
            print("cluster_id %s" % cluster_id)
            curr_image_cluster_id = cluster_id
            encodings.get(cluster_id).append(fe)
            action_taken = True

    if not action_taken:
        curr_image_cluster_id = "cluster_%s" % (len(encodings.keys()) + 1)
        print("creating new cluster %s" % curr_image_cluster_id)
        encodings[curr_image_cluster_id] = [fe]
    curr_cluster = os.path.join(results_path, curr_image_cluster_id)
    curr_cluster_dir = Path(curr_cluster)
    curr_cluster_dir.mkdir(parents=True, exist_ok=True)
    copyfile(filepath, os.path.join(curr_cluster_dir, file))

In [104]:
curr = 0
for subdir, dirs, files in os.walk(p):
    total = len(files)
    for file in files:
        filepath = os.path.join(subdir, file)
        print("File: %s" % filepath)
        process_file(filepath)
        curr += 1
        print("file %s/%s - %s encodings" % (curr, total, len(encodings)))
        print()

File: /Users/rashisolanki/Downloads/face-clustering/dataset/8.png
creating new cluster cluster_1
file 1/25 - 1 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/9.png
results [False] cluster_1
creating new cluster cluster_2
file 2/25 - 2 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/14.png
results [True] cluster_1
cluster_id cluster_1
results [True] cluster_2
cluster_id cluster_2
file 3/25 - 2 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/15.png
results [True, True] cluster_1
cluster_id cluster_1
results [False, True] cluster_2
file 4/25 - 2 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/17.png
results [False, False, False] cluster_1
results [False, False] cluster_2
creating new cluster cluster_3
file 5/25 - 3 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/16.png
results [False, False, False] cluster_1
results [False, False] cluster_2
results [False] cluster_3
creating

In [93]:
print("There are %s different people" % len(encodings.keys()))

There are 1 different people


In [19]:
# Create a tkinter root window for the file dialog
root = tk.Tk()
root.withdraw()  

user_input_image_path = filedialog.askopenfilename(title="Choose an input image file", filetypes=[("Image files", "*.jpg *.jpeg *.png *.gif")])

if not user_input_image_path:
    print("No input image selected. Exiting.")
else:
    email_id = input("Enter your email ID: ")  

    if not email_id:
        print("No email ID provided. Exiting.")
    else:
        process_user_input_image(user_input_image_path, email_id)

2023-10-11 11:51:07.065 python[55062:704547] +[CATransaction synchronize] called within transaction
2023-10-11 11:51:07.080 python[55062:704547] +[CATransaction synchronize] called within transaction
2023-10-11 11:51:10.387 python[55062:704547] +[CATransaction synchronize] called within transaction


Enter your email ID: rashimahavir.solanki2020@vitstudent.ac.in
The detected face belongs to cluster: cluster_6
Email sent successfully.


In [ ]:
root.destroy()